In [1]:
import tensorflow as tf
import numpy as np
import progressbar
import os


### Run the following cell only once in the beginning###
It will set up the computational graph of the RNN network.
Parameters of the RNN can be adjusted at the bottom of the cell. Change to whatever value 
you deem useful, then run the cell.

In [17]:
 class RNNwavefunction(object):
    def __init__(self,systemsize,scope='RNNwavefunction',homogeneous=True):
        """
            systemsize:  int
                         number of sites
            cell:        a tensorflow RNN cell
            units:       list of int
                         number of units per RNN layer           
            scope:       str
                         the name of the name-space scope
            homogeneous: bool
                         True: use the same RNN cell at each
                         False: use a different RNN cell at each site
        """
        self.graph=tf.Graph()
        self.scope=scope
        self.N=systemsize
        self.homogeneous=homogeneous
#         with self.graph.as_default():
#             with tf.variable_scope(self.scope,reuse=tf.AUTO_REUSE):
#                 if homogeneous:
#                     self.lstm=[tf.nn.rnn_cell.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(cell(units[n],activation=activation,name='LSTM_{0}'.format(n)),input_keep_prob=keepprob,output_keep_prob=0.5) for n in range(len(units))])]
#                 else:
#                     self.lstm=[tf.nn.rnn_cell.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(cell(units[n],activation=activation,name='LSTM_{0}'.format(n)),input_keep_prob=keepprob,output_keep_prob=0.5) for n in range(len(units))])]*self.N

    def sample(self,numsamples,inputdim,keepprob,cell=tf.contrib.rnn.LSTMCell,activation=tf.nn.relu,units=[10]):
        """
            generate samples from a probability distribution parametrized by a recurrent network
            ------------------------------------------------------------------------
            Parameters: 
            
            numsamples:      int
                             number of samples to be produced
            inputdim:        int
                             hilbert space dimension
    
            ------------------------------------------------------------------------
            Returns:         a tuple (samples,log-probs)
             
            samples:         tf.Tensor of shape (numsamples,systemsize)
                             the samples in integer encoding
            log-probs        tf.Tensor of shape (numsamples,)
                             the log-probability of each sample 
        """
        with self.graph.as_default():
            with tf.variable_scope(self.scope,reuse=tf.AUTO_REUSE):
                if self.homogeneous:
                    self.lstm=[tf.nn.rnn_cell.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(cell(units[n],activation=activation,name='LSTM_{0}'.format(n)),input_keep_prob=keepprob,output_keep_prob=0.5) for n in range(len(units))])]
                else:
                    self.lstm=[tf.nn.rnn_cell.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(cell(units[n],activation=activation,name='LSTM_{0}'.format(n)),input_keep_prob=keepprob,output_keep_prob=0.5) for n in range(len(units))])]*self.N

                b=np.zeros((numsamples,inputdim)).astype(np.float32)
                b[:,0]=np.ones(numsamples)
                inputs=tf.constant(dtype=tf.float32,value=b,shape=[numsamples,inputdim])
                self.inputdim=inputs.shape[1]
                self.outputdim=self.inputdim
                self.numsamples=inputs.shape[0]
                samples=[]
                one_hot_samples=[]
                probs=[]
                lstm_state=self.lstm[0].zero_state(inputs.shape[0],dtype=tf.float32)
                if not self.homogeneous:
                    lstm_output, lstm_state = self.lstm[0](inputs, lstm_state)
                    output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(0))
                    probs.append(output)
                    temp=tf.reshape(tf.multinomial(tf.log(output),num_samples=1),[-1,])
                    samples.append(temp)
                    inputs2=tf.one_hot(temp,depth=self.outputdim)
                    one_hot_samples.append(inputs2)

                    for n in range(1,self.N):
                        lstm_output, lstm_state = self.lstm[n](inputs2, lstm_state)
                        output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(n))
                        probs.append(output)
                        temp=tf.reshape(tf.multinomial(tf.log(output),num_samples=1),[-1,])
                        samples.append(temp)
                        inputs2=tf.one_hot(temp,depth=self.outputdim)
                        one_hot_samples.append(inputs2)


                else:
                    lstm_output, lstm_state = self.lstm[0](inputs, lstm_state)
                    output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(0))
                    probs.append(output)
                    temp=tf.reshape(tf.multinomial(tf.log(output),num_samples=1),[-1,])
                    samples.append(temp)
                    inputs2=tf.one_hot(temp,depth=self.inputdim)
                    one_hot_samples.append(inputs2)

                    for n in range(1,self.N):
                        lstm_output, lstm_state = self.lstm[0](inputs2, lstm_state)
                        output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(n))
                        probs.append(output)
                        temp=tf.reshape(tf.multinomial(tf.log(output),num_samples=1),[-1,])
                        samples.append(temp)
                        inputs2=tf.one_hot(temp,depth=self.outputdim)
                        one_hot_samples.append(inputs2)

        self.samples=tf.stack(values=samples,axis=1)
        one_hot_samples=tf.transpose(tf.stack(values=one_hot_samples,axis=2),perm=[0,2,1])
        temp=tf.transpose(tf.stack(values=probs,axis=2),perm=[0,2,1])
        #mask=tf.greater(one_hot_samples,0.0001)
        #zeros = tf.zeros_like(temp)
        #self.log_probs=tf.reduce_sum(tf.log(tf.reduce_sum(tf.where(mask,temp,zeros),axis=2)),axis=1)
        self.log_probs=tf.reduce_sum(tf.log(tf.reduce_sum(tf.multiply(temp,one_hot_samples),axis=2)),axis=1)
        return self.samples,self.log_probs
    
    def probability(self,samples,inputdim,keepprob,cell=tf.contrib.rnn.LSTMCell,activation=tf.nn.relu,units=[10]):
        """
            calculate the log-probabilities of ```samples``
            ------------------------------------------------------------------------
            Parameters: 
            
            samples:         tf.Tensor
                             a tf.placeholder of shape (number of samples,system-size) 
                             containing the input samples in integer encoding
            inputdim:        int
                             dimension of the input space
                      
            ------------------------------------------------------------------------         
            Returns: 
            log-probs        tf.Tensor of shape (number of samples,)
                             the log-probability of each sample         
            """
        with self.graph.as_default():

            self.inputdim=inputdim
            self.outputdim=self.inputdim

            self.numsamples=samples.shape[0]
            b=np.zeros((self.numsamples,self.inputdim)).astype(np.float32)
            b[:,0]=np.ones(self.numsamples)
            inputs=tf.constant(dtype=tf.float32,value=b,shape=[self.numsamples,self.inputdim])
        
            with tf.variable_scope(self.scope,reuse=tf.AUTO_REUSE):
                if self.homogeneous:
                    self.lstm=[tf.nn.rnn_cell.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(cell(units[n],activation=activation,name='LSTM_{0}'.format(n)),input_keep_prob=keepprob,output_keep_prob=0.5) for n in range(len(units))])]
                else:
                    self.lstm=[tf.nn.rnn_cell.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(cell(units[n],activation=activation,name='LSTM_{0}'.format(n)),input_keep_prob=keepprob,output_keep_prob=0.5) for n in range(len(units))])]*self.N
                
                probs=[]

                lstm_state=self.lstm[0].zero_state(self.numsamples,dtype=tf.float32)
                if not self.homogeneous:
                    lstm_output, lstm_state = self.lstm[0](inputs, lstm_state)
                    output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(0))
                    probs.append(output)
                    inputs2=tf.reshape(tf.one_hot(tf.slice(samples,begin=[np.int32(0),np.int32(0)],size=[np.int32(-1),np.int32(1)]),depth=self.outputdim),shape=[self.numsamples,self.inputdim])
                    
                    for n in range(1,self.N):
                        lstm_output, lstm_state = self.lstm[n](inputs2, lstm_state)
                        output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(n))
                        probs.append(output)
                        inputs2=tf.reshape(tf.one_hot(tf.reshape(tf.slice(samples,begin=[np.int32(0),np.int32(n)],size=[np.int32(-1),np.int32(1)]),shape=[self.numsamples]),depth=self.outputdim),shape=[self.numsamples,self.inputdim])
            
                else:
                    lstm_output, lstm_state = self.lstm[0](inputs, lstm_state)
                    output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(0))
                    probs.append(output)
                    inputs2=tf.reshape(tf.one_hot(tf.slice(samples,begin=[np.int32(0),np.int32(0)],size=[np.int32(-1),np.int32(1)]),depth=self.outputdim),shape=[self.numsamples,self.inputdim])
                    
                    for n in range(1,self.N):
                        lstm_output, lstm_state = self.lstm[0](inputs2, lstm_state)
                        output=tf.layers.dense(lstm_output,self.outputdim,activation=tf.nn.softmax,name='wf_dense_{0}'.format(n))
                        probs.append(output)
                        inputs2=tf.reshape(tf.one_hot(tf.reshape(tf.slice(samples,begin=[np.int32(0),np.int32(n)],size=[np.int32(-1),np.int32(1)]),shape=[self.numsamples]),depth=self.outputdim),shape=[self.numsamples,self.inputdim])
            
            temp=tf.transpose(tf.stack(values=probs,axis=2),perm=[0,2,1])
            one_hot_samples=tf.one_hot(samples,depth=self.inputdim)
            #mask=tf.greater(one_hot_samples,0.001)
            #zeros = tf.zeros_like(temp)
            #self.log_probs=tf.reduce_sum(tf.log(tf.reduce_sum(tf.where(mask,temp,zeros),axis=2)),axis=1)
            self.log_probs=tf.reduce_sum(tf.log(tf.reduce_sum(tf.multiply(temp,one_hot_samples),axis=2)),axis=1)
            return self.log_probs


units=[300]#list containing the number of hidden units for each layer of the networks
N=8
input_dim=2
numsamples=20 #only for initialization; later I'll use a much larger value (see below)
#cell=tf.contrib.rnn.LSTMCell()
#wf=RNNwavefunction(N,units=units,cell=tf.contrib.rnn.GRUCell,activation=tf.keras.activations.linear) #contains the graph with the RNNs
wf=RNNwavefunction(N)

with wf.graph.as_default():#inserting a few placeholders into the graph
    inputs=tf.placeholder(dtype=tf.int32,shape=[numsamples,N])
    learningrate=tf.placeholder(dtype=tf.float32,shape=[])
    keepprob=tf.placeholder(dtype=tf.float32,shape=[])
sampling=wf.sample(numsamples,input_dim,keepprob,cell=tf.contrib.rnn.LSTMCell,activation=tf.keras.activations.linear) #call this function once to create the dense layers
with wf.graph.as_default(): #now initialize everything 
    probs=wf.probability(inputs,input_dim,keepprob)
    optimizer=tf.train.AdamOptimizer(learning_rate=learningrate)
    init=tf.global_variables_initializer()


### Initialization of the RNN ###
run the cell below only once, otherwise you will reinitialize the network

In [3]:
#with wf.graph.as_default():
sess=tf.Session(graph=wf.graph)
sess.run(init)

The following cell generates samples from the LSTM, together with their log-probabilites

In [4]:
generated_samples,generated_log_probs=sess.run(sampling,feed_dict={keepprob: 1.0}) #a tests
print(generated_log_probs)


[-5.7172027 -5.3007636 -5.730796  -5.638402  -4.881246  -5.7099276
 -5.3763647 -5.3637514 -5.462563  -5.044448  -6.2424517 -6.037634
 -6.069004  -5.5766478 -5.574834  -5.32192   -5.193973  -4.981885
 -5.518626  -5.35106  ]


The following cell takes a bunch of input-samples an calculates their log-probabilities. The output of the 
cell should be identical to the output of the previous one (I'm calculating the probabilities of the generated samples again).

In [5]:
samples=np.random.randint(0,2,size=(numsamples,N))
sample_log_probs=sess.run(probs,feed_dict={inputs:generated_samples,keepprob:1.0})
print(sample_log_probs)


[-5.6692047 -5.5594597 -5.696536  -5.909798  -5.288148  -4.822653
 -5.083637  -5.063472  -5.712101  -6.054822  -5.721218  -5.5354805
 -5.810268  -5.8715296 -5.998554  -5.3953433 -5.3916216 -5.7642927
 -5.212789  -5.925873 ]


In [6]:
with wf.graph.as_default():
    variables_names =[v.name for v in tf.trainable_variables()]
    print(variables_names)
    values = sess.run(variables_names)
    for k,v in zip(variables_names, values):
        print(k,v[0])
        break

['RNNwavefunction/multi_rnn_cell/cell_0/LSTM_0/kernel:0', 'RNNwavefunction/multi_rnn_cell/cell_0/LSTM_0/bias:0', 'RNNwavefunction/wf_dense_0/kernel:0', 'RNNwavefunction/wf_dense_0/bias:0', 'RNNwavefunction/wf_dense_1/kernel:0', 'RNNwavefunction/wf_dense_1/bias:0', 'RNNwavefunction/wf_dense_2/kernel:0', 'RNNwavefunction/wf_dense_2/bias:0', 'RNNwavefunction/wf_dense_3/kernel:0', 'RNNwavefunction/wf_dense_3/bias:0', 'RNNwavefunction/wf_dense_4/kernel:0', 'RNNwavefunction/wf_dense_4/bias:0', 'RNNwavefunction/wf_dense_5/kernel:0', 'RNNwavefunction/wf_dense_5/bias:0', 'RNNwavefunction/wf_dense_6/kernel:0', 'RNNwavefunction/wf_dense_6/bias:0', 'RNNwavefunction/wf_dense_7/kernel:0', 'RNNwavefunction/wf_dense_7/bias:0']
RNNwavefunction/multi_rnn_cell/cell_0/LSTM_0/kernel:0 [ 0.07476813  0.30780047  0.03343037 -0.0266746   0.29104358 -0.24790308
  0.07487437 -0.09495552  0.04957685 -0.2898727   0.08468324  0.17791748
 -0.17019305  0.28364658 -0.17864157 -0.29150352 -0.23419428 -0.04210275
 -0.10

### Sampling local energy

$\langle \Psi |H|\Psi \rangle=\sum_{\sigma,\sigma'} \psi_{\sigma'} H_{\sigma'\sigma}\psi_{\sigma}
=\sum_{\sigma'} |\psi_{\sigma'}|^2\sum_{\sigma} H_{\sigma'\sigma}\frac{\psi_{\sigma}}{\psi_{\sigma'}}$

$\partial_{a}\langle \Psi |H|\Psi \rangle=\sum_{\sigma,\sigma'} (\partial_{a}\psi_{\sigma'})H_{\sigma'\sigma}\psi_{\sigma} + \psi_{\sigma'}H_{\sigma'\sigma}(\partial_{a}\psi_{\sigma})=2\sum_{\sigma,\sigma'} (\partial_{a}\psi_{\sigma'})H_{\sigma'\sigma}\psi_{\sigma}=
2\sum_{\sigma,\sigma'}|\psi_{\sigma'}|^2 \frac{(\partial_{a}\psi_{\sigma'})}{\psi_{\sigma'}}H_{\sigma'\sigma}\frac{\psi_{\sigma}}{\psi_{\sigma'}}=$

$\sum_{\sigma'}|\psi_{\sigma'}|^2 \frac{\partial_{a}\psi_{\sigma'}}{\psi_{\sigma'}}2\sum_{\sigma}H_{\sigma'\sigma}\frac{\psi_{\sigma}}{\psi_{\sigma'}}$

The cell below implements the above stuff (not very efficiently)

In [12]:
def XXZMatrixElemets(Jz,Jp,Bz,sigmap):
    """
    computes the matrix element of the periodic XXZ Hamiltonian for a given state sigmap
    -----------------------------------------------------------------------------------
    Parameters:
    Jz, Jp, Bz: np.ndarray of shape (N), (N) and (N), respectively, and dtype=float:
                XXZ parameters
    sigmap:     np.ndarrray of dtype=int and shape (N)
                spin-state, integer encoded (using 0 for down spin and 1 for up spin)
    -----------------------------------------------------------------------------------            
    Returns: 2-tuple of type (np.ndarray,np.ndarray)
             sigmas:         np.ndarray of dtype=int and shape (?,N)
                             the states for which there exist non-zero matrix elements for given sigmap
             matrixelements: np.ndarray of dtype=float and shape (?)
                             the non-zero matrix elements
    """
    sigmas=[]
    matrixelements=[]
    N=len(Bz)
    #the diagonal part is simply the sum of all Sz-Sz interactions plus a B field
    diag=np.dot(sigmap-0.5,Bz)
    
    for site in range(N):
        if sigmap[site]!=sigmap[(site+1)%N]:
            diag-=0.25*Jz[site]
        else:
            diag+=0.25*Jz[site]
    matrixelements.append(diag)
    sigmas.append(np.copy(sigmap))
    
    #off-diagonal part:
    for site in range(N):
        if sigmap[site]!=sigmap[(site+1)%N]:
            sig=np.copy(sigmap)
            sig[site]=sig[(site+1)%N]
            sig[(site+1)%N]=sigmap[site]
            sigmas.append(sig)
            matrixelements.append(Jp[site]/2)
    return np.array(sigmas),np.array(matrixelements)

def XXZLocalEnergy(Jz,Jp,Bz,sigmap,RNN):
    """
    DEPRECATED
    computes the local energy for the XXZ model:
    ---------------------------------------------------------------------------------
    Parameters:
    Jz, Jp, Bz: np.ndarray of shape (N-1), (N-1) and (N), respectively, and dtype=float:
                XXZ parameters
    sigmap:     np.ndarrray of dtype=int and shape (N)
                spin-state, integer encoded (using 0 for down spin and 1 for up spin)
    RNN:        fully initialized RNNwavefunction object
    ----------------------------------------------------------------------------------
    Returns:
    the local energy (float) for sigmapp
    """
    sigmas,H=XXZMatrixElemets(Jz,Jp,Bz,sigmap)#note that sigmas[0,:]==sigmap
    with RNN.graph.as_default():
        inputs=tf.placeholder(dtype=tf.int32,shape=[len(sigmas),len(Bz)])
        probs=RNN.probability(inputs,inputdim=2)
        probabilities=sess.run(probs,feed_dict={inputs:sigmas})
    
    return H.dot(probabilities)/probabilities[0]

def XXZLocalEnergies(Jz,Jp,Bz,sigmasp,RNN):
    """
    computes the local energies for the periodic XXZ model for a given spin-state sample sigmasp:
    Eloc(\sigma')=\sum_{sigma} H_{\sigma'\sigma}\psi_{\sigma}/\psi_{\sigma'}
    ----------------------------------------------------------------------------
    Parameters:
    Jz, Jp, Bz: np.ndarray of shape (N), (N) and (N), respectively, and dtype=float:
                XXZ parameters
    sigmasp:    np.ndarrray of dtype=int and shape (numsamples,N)
                spin-states, integer encoded (using 0 for down spin and 1 for up spin)
    RNN:        fully initialized RNNwavefunction object
    ----------------------------------------------------------------------------
    Returns:
    np.ndarray of shape (numsamples)and dtype=float containing the local energies for each samples
    """
    sigmas=np.empty((0,len(Bz)))
    H=np.empty(0)
    slices=[]
    for n in range(sigmasp.shape[0]):
        sigmap=sigmasp[n,:]
        temp1,temp2=XXZMatrixElemets(Jz,Jp,Bz,sigmap)#note that sigmas[0,:]==sigmap
        H=np.append(H,temp2)
        slices.append(slice(sigmas.shape[0],sigmas.shape[0]+temp1.shape[0]))
        sigmas=np.append(sigmas,temp1,axis=0)
    with RNN.graph.as_default():
        temp_inputs=tf.placeholder(dtype=tf.int32,shape=[len(sigmas),len(Bz)])
        temp_probs=RNN.probability(temp_inputs,inputdim=2,keepprob=keepprob)
        log_probabilities=sess.run(temp_probs,feed_dict={temp_inputs:sigmas,keepprob:1.0})
    localEnergies=[]
    for n in range(len(slices)):
        s=slices[n]
        localEnergies.append(H[s].dot(np.exp(0.5*(log_probabilities[s]-log_probabilities[s][0]))))
    return np.array(localEnergies)



### A small check to confirm that the matrix elements are correct ###

In [13]:
import itertools
import XXZED

N_=8
basis_=XXZED.binarybasisU1(N_,np.int(N_/2))

basis=np.array([np.array(list(bin(b)[2::].zfill(N_))).astype(int) for b in basis_])

Jz=np.ones(N_)
Jp=-np.ones(N_)
Bz=np.zeros(N_)

H=np.zeros((basis.shape[0],basis.shape[0]))
for n in range(basis.shape[0]):
    sigmas,elements=XXZMatrixElemets(Jz,Jp,Bz,np.reshape(basis[n,:],(N_)))
#     print(basis[n,:])
#     print(sigmas)
#     input()
    for m in range(sigmas.shape[0]):
        for b in range(basis.shape[0]):
            if np.all(basis[b,:]==sigmas[m,:]):
                H[b,n]=elements[m]
eta,U=np.linalg.eigh(H)
#print(np.nonzero(eta==np.min(eta))[0][0])
print('the follwing numbers should all be > 0')
print(U[:,np.nonzero(eta==np.min(eta))[0][0]])

the follwing numbers should all be > 0
[0.00746966 0.03474209 0.05454486 0.03474209 0.00746966 0.05454486
 0.16440628 0.13713385 0.03474209 0.09005865 0.16440628 0.05454486
 0.05454486 0.03474209 0.00746966 0.03474209 0.13713385 0.16440628
 0.05454486 0.16440628 0.39829674 0.16440628 0.16440628 0.13713385
 0.03474209 0.05454486 0.16440628 0.09005865 0.13713385 0.16440628
 0.05454486 0.03474209 0.05454486 0.03474209 0.00746966 0.00746966
 0.03474209 0.05454486 0.03474209 0.05454486 0.16440628 0.13713385
 0.09005865 0.16440628 0.05454486 0.03474209 0.13713385 0.16440628
 0.16440628 0.39829674 0.16440628 0.05454486 0.16440628 0.13713385
 0.03474209 0.00746966 0.03474209 0.05454486 0.05454486 0.16440628
 0.09005865 0.03474209 0.13713385 0.16440628 0.05454486 0.00746966
 0.03474209 0.05454486 0.03474209 0.00746966]


In [14]:
#print(np.sum(basis,axis=1))
print ('the following number should be ~ -3.65109341: ',np.min(eta))
#confirmed to be correct

the following number should be ~ -3.65109341:  -3.6510934089371783


### Load a checkpointed wave-function ###

In [11]:
import os
path=os.getcwd()
print((path))
with tf.variable_scope(wf.scope,reuse=tf.AUTO_REUSE):
    with wf.graph.as_default():
        saver=tf.train.Saver()
        saver.restore(sess,path+'/RNNwavefunction_N{0}_units{1}.ckpt'.format(N,units))

/home/maga/Dropbox/Work/RNN_wavefunctions
INFO:tensorflow:Restoring parameters from /home/maga/Dropbox/Work/RNN_wavefunctions/RNNwavefunction_N8_units[500].ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /home/maga/Dropbox/Work/RNN_wavefunctions/RNNwavefunction_N8_units[500].ckpt
	 [[Node: save_1/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/home/maga/software/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/maga/software/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/maga/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/maga/software/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line

In [15]:
meanEnergy=[]
varEnergy=[]

In [16]:
path=os.getcwd()
Jz=np.ones(N)
Jp=-np.ones(N)
Bz=np.zeros(N)

#for a given network, generate a large number of samples:
numsamples=20000
lr=np.float32(0.01)
# with tf.variable_scope(wf.scope,reuse=tf.AUTO_REUSE):
#     with wf.graph.as_default():
#         optimizer=tf.train.AdamOptimizer(learning_rate=learningrate)
#         #sess.run(tf.variables_initializer(optimizer.variables()),feed_dict={learningrate: lr})
with tf.variable_scope(wf.scope,reuse=tf.AUTO_REUSE):
    with wf.graph.as_default():
        saver=tf.train.Saver()
        Eloc=tf.placeholder(dtype=tf.float32,shape=[numsamples])
        samp=tf.placeholder(dtype=tf.int32,shape=[numsamples,N])
        log_probs_=wf.probability(samp,inputdim=2,keepprob=keepprob)
        #now calculate the fake cost function:
        cost=tf.reduce_mean(tf.multiply(log_probs_,Eloc)) #factor of 2 in the above equation 
                                          #cancels when taking log(sqrt(prob))=log(sqrt(psi^2))`
                                          #=log(psi)=2*log(psi^2)->log(psi)=1/2*log(psi^2)=1/2*log_probs
        gradients, variables = zip(*optimizer.compute_gradients(cost))
        #clipped_gradients,_=tf.clip_by_global_norm(gradients,1.0)
        optstep=optimizer.apply_gradients(zip(gradients,variables))
        sess.run(tf.variables_initializer(optimizer.variables()),feed_dict={learningrate: lr})
for it in range(1000):
    samples,log_probs=sess.run(wf.sample(numsamples=numsamples,inputdim=2,keepprob=keepprob),feed_dict={keepprob:1})
    local_energies=XXZLocalEnergies(Jz,Jp,Bz,samples,wf)
    meanE=np.mean(local_energies)
    varE=np.var(local_energies)
    meanEnergy.append(meanE)   
    varEnergy.append(varE)
    print('mean(E): {0} \pm {1}'.format(meanE,np.sqrt(varE)))
    with tf.variable_scope(wf.scope,reuse=tf.AUTO_REUSE):
        with wf.graph.as_default():

            sess.run(optstep,feed_dict={Eloc:local_energies,samp:samples,learningrate: lr,keepprob:1.0})
            if it%10==0:
                saver.save(sess,path+'/RNNwavefunction_N{0}_units{1}.ckpt'.format(N,units))






mean(E): -1.9863784604728223 \pm 1.4491462665157018
mean(E): -2.004730501356721 \pm 1.4334951565727612
mean(E): -2.003879830124974 \pm 1.4449950229400343
mean(E): -2.0025246302694084 \pm 1.4343775697986925
mean(E): -1.9839182492166758 \pm 1.4239380651922435
mean(E): -1.9967129244819284 \pm 1.414758551645511
mean(E): -2.0282533726483583 \pm 1.412893688232432
mean(E): -2.0195562712401154 \pm 1.4142660043841133
mean(E): -2.0228245173320176 \pm 1.428550507077093
mean(E): -2.003485303504765 \pm 1.4097282911273208
mean(E): -2.026028910920024 \pm 1.413109418040019
mean(E): -2.040016864052415 \pm 1.4112268825573804
mean(E): -2.022483886407316 \pm 1.4145885145371155
mean(E): -2.0562956987947225 \pm 1.4149746120959381
mean(E): -2.0506620590150355 \pm 1.3950235268002105


KeyboardInterrupt: 

In [23]:
a=[1,2,3]
bl=''
for n in a:
    bl+=str(n)
print(bl)


123
